In [20]:
import pandas as pd

In [21]:
edstay_file = pd.read_csv('../raw_data/edstay_encounters.csv')
medrecon_file = pd.read_csv('../raw_data/medrecon.csv')

In [22]:
# get 200 most occuring ndcs
ncds = medrecon_file['ndc'].value_counts().head(200).index.tolist()

# convert ndc to "other" if not in top 200
medrecon_file['ndc'] = medrecon_file['ndc'].apply(lambda x: x if x in ncds else 'other')

In [23]:
# group ndc by stay_id, then put into a list
ndcs = medrecon_file.groupby('stay_id')['ndc'].apply(list).reset_index()

In [24]:
edstay_file = edstay_file.join(ndcs.set_index('stay_id'), on='stay_id')

In [25]:
# handle nan
edstay_file['ndc'] = edstay_file['ndc'].apply(lambda x: x if type(x) == list else [])

# remove duplicates in ndc lists
edstay_file['ndc'] = edstay_file['ndc'].apply(lambda x: list(set(x)))

# get the longest list of ndcs
edstay_file['ndc'].apply(lambda x: len(x)).max()

31

In [26]:
# sort ndc lists according to rank of ndc in top 200, put "other" at the end
edstay_file['ndc'] = edstay_file['ndc'].apply(lambda x: sorted(x, key=lambda y: ncds.index(y) if y != 'other' else 999))

In [27]:
# get longest list
edstay_file['ndc'].apply(lambda x: len(x)).max()

31

In [35]:
uniqe_ndcs = edstay_file['ndc'].dropna().explode().unique()

# remove nan
uniqe_ndcs = [x for x in uniqe_ndcs if (type(x) == str or type(x) == int)]

In [37]:
one_hot_ndc = pd.DataFrame(columns=uniqe_ndcs)

for ndc in uniqe_ndcs:
    one_hot_ndc[ndc] = edstay_file['ndc'].apply(lambda x: 1 if ndc in x else 0)

In [39]:
edstay_file = edstay_file.join(one_hot_ndc)

In [41]:
edstay_file.shape

(425087, 225)

In [40]:
edstay_file.to_csv('../processed_data/edstay_encounters_with_ndc.csv', index=False)